# Setup

In [2]:
#https://www.kaggle.com/piotrstanczyk/gfootball-template-bot  G-FOOTBALL TEMPLATE BOT
# Install:
# Kaggle environments.
#!git clone https://github.com/Kaggle/kaggle-environments.git
#!cd kaggle-environments && pip install .

# GFootball environment.
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v5 https://github.com/Georgemouts/football.git #https://github.com/google-research/football.git  
!mkdir -p football/third_party/gfootball_engine/lib

!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.8.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

!pip install utils
#!pip install learning

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.2 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [742 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-sec

# All the imports

In [3]:
#from gfootball.env.football_env import FootballEnv
#from kaggle_environments import make
from gfootball.env.config import Config
import gfootball.env as football_env

#import dqn libraries
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
import numpy as np
import pandas as pd
import itertools
import utils
import random
from collections import deque
import matplotlib.pyplot as plt
#from utils import plot_learning_curve
#import env 
import gym
import gfootball 
import math

In [4]:
env = football_env.create_environment(env_name ='academy_empty_goal',render=False,representation='simple115v2')

new_observation,reward,done,info = env.step(5)
print(new_observation)

[ 0.         -0.          0.         -0.          0.02       -0.
  0.11059734 -0.          0.         -0.00191829]


# Initialize variables

In [19]:
BUFFERSIZE = 10000  #how many experiences will store |ReplayBufferSize=100,000
REWBUFFERSIZE = 500   #how many episode rewards will store|RewardBufferSize =10,000
MINREPLAYSIZE= 1000 # Episode is 3000 steps
GAMMA = 0.99


EPSILON_START=1.0
EPSILON_END =0.01
EPSILON_DECAY=10000

TARGET_UPDATE_FREQ = 400 # target parameters will be equal to online parameters
BATCH_SIZE = 32


# Deep Q Network Model

# Agent 

In [ ]:
class Agent() : 
  def __init__(self,gamma,epsilon):
    self.gamma =gamma
    self.epsilon =epsilon
    
    self.ReplayBuffer = deque(maxlen=BUFFERSIZE) #Store experiences 
    self.RewBuffer = deque(maxlen=REWBUFFERSIZE) #Store rewards
    
    self.online_net = DQN(env)   
    self.target_net = DQN(env)  

    self.target_net.load_state_dict(self.online_net.state_dict())


  def transition(self,obs,new_obs,action,reward ,done): # should be tuple ? 
    self.obs =obs
    self.action=action
    self.reward =reward 
    self.done=done
    #self.info =info
    self.new_obs =new_obs
    TransitionTuple= (obs,new_obs,action,reward,done)
    #print("class",TransitionTuple)
    return TransitionTuple

  def learn(self,loss):
    
    self.loss=loss
    self.optimizer=torch.optim.Adam(self.online_net.parameters(),lr =1e-3) 
    self.optimizer.zero_grad()
    self.loss.backward()
    self.optimizer.step()

  def Arrays_To_Tensors(self,transitions): #make arrays -> pytorch tensors
    self.transitions=transitions

    #Store observations as arrays
    obses = np.asarray([t[0] for t in transitions])
    new_obses = np.asarray([t[1]for t in transitions])
    actions = np.asarray([t[2] for t in transitions])
    rewards = np.asarray([t[3] for t in transitions])
    dones = np.asarray([t[4] for t in transitions])
  
    #Convert observation arrays to pytorch tensors
    obses_t=torch.as_tensor(obses,dtype=torch.float32)
    actions_t = torch.as_tensor(actions,dtype=torch.int64).unsqueeze(-1) #making batch dimension to one dimension
    rewards_t = torch.as_tensor(rewards,dtype= torch.float32).unsqueeze(-1)
    dones_t = torch.as_tensor(dones,dtype= torch.float32).unsqueeze(-1)
    new_obses_t = torch.as_tensor(new_obses,dtype= torch.float32)

    return obses_t,actions_t,rewards_t,dones_t,new_obses_t




# All Prints

In [ ]:
class All_prints():
  
  def __init___(self,step):
    self.step=step
    #self.RewBuffer = RewBuffer
    self.reward=reward

  def printstats(self,step,RewBuffer,eps_reward,info):  #Kaleitai otan ginei done , diladi otan teleiosei ena paixnidi
    self.step=step
    self.RewBuffer=RewBuffer
    self.eps_reward=eps_reward
    self.info=info
    print("-->Episode:",self.step%3000 + 1,"\t",step,"\t","Episode Reward:",self.eps_reward,"<--")
    #print("Step",step)
    print("lista apo rewards mexri tora" ,self.RewBuffer)
    print("Avg reward", np.mean(self.RewBuffer))
    print("---------------------------------------------------")

  def print_who_scored(self, reward):
    self.reward=reward
    if(self.reward==1):
      print("our team scored !!!")
    elif(self.reward ==-1):
      print("opponent team scored")
    
    
 
  def rew_graph(self,RewBuffer,step,num_of_eps):
      self.RewBuffer=RewBuffer
      self.step=step
      self.num_of_eps=num_of_eps
      episode=self.step%3000
      eps_list=list(range(1,self.num_of_eps+1))#pairnei to proto , den pairnei to teleytaio
      #print(agent.RewBuffer,eps_list)
      plt.plot(eps_list,self.RewBuffer)
      plt.xlabel('Episode')
      plt.ylabel('Rewards')
      plt.grid(True)
      plt.show()

# Custom Rewards

In [ ]:
class Custom_Rewards():
  def __init__(self):
    self.obs=obs    
    #self.action=action
    #self.reward =reward
  def custom_rew(self,obs,action,reward,new_obs):
    
    self.obs=obs
    self.action = action
    self.reward = reward
    self.new_obs=new_obs
    
    
#if the next action is shooting the ball, the reward will gain 0.2 point ||
#an exo ti mpala kai einai se apostasi x > 0.5 sto gipedo kai exei kanei shout einai kalo    
    if(self.obs[95] == 1 and self.obs[88] > 0.6 and self.action == 12):
      self.reward += 0.1
      print("EKANE SHOUT MPROSTA APO TO KENTRO ,+0.1")
      return self.reward

    #An kanei shout piso apo to kentro na xanei ligo
    elif(self.obs[95]==1 and self.obs[88] < 0 and self.action == 12):
      self.reward -= 0.05
      #print("EKANE SHOUT PISO APO TO KENTRO, -0.05")
      return self.reward

# if we steal the ball, the reward will gain 0.05 points
    elif self.obs[96] == 1 and self.new_obs[95] == 1:  #obs[96] ball owner right team / obs[95] ball owner left team
        self.reward += 0.05
        #print("PHRE THN MPALA APO TON ANTIPALO , +0.05")
        return self.reward
#if we lose the ball, the reward will lose 0.05 points
    elif self.obs[95] == 1 and self.new_obs[96] == 1:
        self.reward -=0.02
        #print("PHRE THN MPALA O ANTIPALOS , -0.02")
        return self.reward
    else:
      return self.reward


#TODO: if active player far from the ball, the reward will lose by the moving distance

In [30]:
class DQN(nn.Module):
  def __init__(self,env):
    super().__init__()
    #input_dims = int(np.prod(env.observation_space.shape)) #neurons input layer = number of observations
    self.net =nn.Sequential(
                            nn.Linear(10,250), 
                            #nn.ReLU(),
                            nn.ReLU(),
                            nn.Linear(250,4)) # neurons outpul layer = number of actons
                            
  def forward(self,x):
    return self.net(x) #use the dqnetwork

  def act(self,obs): #returns the best acrtion/highest value action of the net 
    obs_t =torch.as_tensor(obs,dtype=torch.float32)
    q_values=self(obs_t.unsqueeze(0)) # make tensor a batch dimension

    max_q_index = torch.argmax(q_values,dim=1)[0]   # taking action with highest q value
    action = max_q_index.detach().item() # making tensor to integer which represents action 
    #print("QVALUES",q_values)
    #print("MAX_VALUE",max_q_index)

    return action 



# Create Environment -- Main 


In [41]:
env = football_env.create_environment(env_name ='academy_empty_goal',render=False,representation='simple115v2') #List with the 115 states 

replay_buffer =deque(maxlen=BUFFERSIZE)
rew_buffer = deque([0,0],maxlen=100)
episode_reward = 0.0


episode_reward =0.0
online_net=DQN(env)
target_net=DQN(env)

target_net.load_state_dict(online_net.state_dict())
optimizer = torch.optim.Adam(online_net.parameters() , lr =0.001)


Action_list=[4,5,6,12]

#Initialize replaybuffer 
obs = env.reset()

for i in range(MINREPLAYSIZE):
  action = random.randint(0,3) # random action 
  while((obs[0]<0.65)  and (Action_list[action]==12)):
    action = random.randint(0,2)

  new_obs ,reward,done ,info = env.step(Action_list[action])



  transition = (obs, action,reward ,done ,new_obs)
  replay_buffer.append(transition)
  obs = new_obs

  if (done ==1):
    obs = env.reset()

#Main Training loop
obs =env.reset()

for step in itertools.count():
    epsilon = np.interp(step,[0,EPSILON_DECAY],[EPSILON_START , EPSILON_END]) # from 100%random actions to 1% random actions
    rnd_sample = random.random()

    if rnd_sample<epsilon :
        action = random.randint(0,3) # random action 
        while((obs[0]<0.65)  and (Action_list[action]==12)):
            action = random.randint(0,2)
            #print("Epilego action=",action)
    else:
        action = online_net.act(obs)    #best action
        #while((obs[0]<0.65)  and (Action_list[action]==12)): #Den kanei shout ektos periohis
        #action = online_net.act(obs)
    
  #custom actions - here 
    if(done ==1 and reward != 1): #if ball is out ,loses -2
        print("Ball is out reward:",reward)
        reward = reward -10
        print("ball is out -10","episode",i,"Ball Position",obs[4],obs[5],obs[6])
    if(reward==1 and  done ==1): #if agent scores , wins +5
        print("goal","episode",i,)
        reward = 10

    if((obs[0]<0.5)  and (Action_list[action]==12)): #an shoutarei prin th megali perioxh -2
      
        reward= reward -200
        done=1
        print("shout ektos periohis","episode",i)
      
    if((obs[0]>0.6) and (Action_list[action]==12)): #an shoutarei mesa ti megali periohi +0.1
      #reward= reward +0.1
      print("shout entos periohis Ball Position",obs[4],obs[5],obs[6],"episode",i,"step=",step)
  #custom rew - here 

    new_obs ,reward,done ,_ = env.step(Action_list[action])

    transition = (obs, action ,reward ,done, new_obs)
  
    replay_buffer.append(transition) # gemizei me transition
 
    obs = new_obs
    episode_reward += reward


    if (done ==1):
        print("Number of steps" ,step,"Reward:",episode_reward)
        obs = env.reset()
        rew_buffer.append(episode_reward)
        episode_reward =0.0

    #Start Gradient Step
    transitions = random.sample(replay_buffer,BATCH_SIZE) #trabao tyxaia transitions apo to replay buffer
  
  
    obses = np.asarray([t[0] for t in transitions])
    actions = np.asarray([t[1] for t in transitions])
    rewards = np.asarray([t[2] for t in transitions])
    dones = np.asarray([t[3] for t in transitions])
    new_obses = np.asarray([t[4] for t in transitions])

    obses_t = torch.as_tensor(obses, dtype=torch.float32)
    actions_t = torch.as_tensor(actions, dtype=torch.int64).unsqueeze(-1)
    rewards_t = torch.as_tensor(rewards, dtype=torch.float32).unsqueeze(-1)
    dones_t = torch.as_tensor(dones, dtype=torch.float32).unsqueeze(-1)
    new_obses_t = torch.as_tensor(new_obses, dtype=torch.float32)

  #Compute Targets
    target_q_values = target_net(new_obses_t)
    max_target_q_values=target_q_values.max(dim=1,keepdim=True)[0]

    targets = rewards_t *GAMMA *(1-dones_t) * max_target_q_values

  #Compute Loss
    q_values = online_net(obses_t)
    action_q_values =torch.gather(input = q_values,dim=1,index=actions_t) # take the q value of the action we did throught the transition

    loss =nn.functional.smooth_l1_loss(action_q_values , targets)

  #Gradient Descent 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  #Update Target Network
    if step % TARGET_UPDATE_FREQ ==0:
        target_net.load_state_dict(online_net.state_dict())


    if step %1000 == 0 and step!=0:
        print("Step", step)
        print("Avg Reward",np.mean(rew_buffer))

shout entos periohis Ball Position 0.62212974 0.028428761 0.11023999 episode 999 step= 99
shout entos periohis Ball Position 0.8702313 -0.018912641 0.12332839 episode 999 step= 110
Number of steps 115 Reward: 1.0
goal episode 999
shout entos periohis Ball Position 0.71764314 -0.0189759 0.12842129 episode 999 step= 228
shout entos periohis Ball Position 0.7234285 -0.023299227 0.119563155 episode 999 step= 229
shout entos periohis Ball Position 0.7979809 -0.040747304 0.48300937 episode 999 step= 234
shout entos periohis Ball Position 0.9487334 -0.045660555 0.48662722 episode 999 step= 237
Number of steps 238 Reward: 0.0
Ball is out reward: 0.0
ball is out -10 episode 999 Ball Position 0.02 -0.0 0.11061639
shout entos periohis Ball Position 0.66387266 -0.0208405 0.11059207 episode 999 step= 340
shout entos periohis Ball Position 0.7308361 -0.00037574678 0.29961324 episode 999 step= 346
shout entos periohis Ball Position 0.83583766 0.013587255 0.4239082 episode 999 step= 348
Number of step

KeyboardInterrupt: ignored

In [ ]:
#Custom rewards - find ball team owner in obs list
env.reset()
obs = env.get_state
print(obs)
#print(obs[0])

<bound method GetStateWrapper.get_state of <GetStateWrapper<SingleAgentRewardWrapper<SingleAgentObservationWrapper<Simple115StateWrapper<FootballEnv<GFootball-11_vs_11_kaggle-simple115v2-v0>>>>>>>


In [ ]:
# env = make("football", debug=True, configuration={"save_video": False, "scenario_name": "11_vs_11_kaggle", "running_in_notebook": True})
# env.render

In [ ]:
# ena =env.reset() 
# #dyo=simple_env.reset()

# #print(ena.shape)
# print(ena[0])#olo to environment
# #pprint.pprint(ena[0])


In [ ]:
# %%writefile submission.py
# from kaggle_environments.envs.football.helpers import *

# ## @human_readable_agent wrapper modifies raw observations 
# # provided by the environment:
# # https://github.com/google-research/football/blob/master/gfootball/doc/observation.md#raw-observations
# # into a form easier to work with by humans.
# # Following modifications are applied:
# # - Action, PlayerRole and GameMode enums are introduced.
# # - 'sticky_actions' are turned into a set of active actions (Action enum)
# #    see usage example below.
# # - 'game_mode' is turned into GameMode enum.
# # - 'designated' field is removed, as it always equals to 'active'
# #    when a single player is controlled on the team.
# # - 'left_team_roles'/'right_team_roles' are turned into PlayerRole enums.
# # - Action enum is to be returned by the agent function.
# @human_readable_agent
# def agent(obs):
#     # Make sure player is running.
#     if Action.Sprint not in obs['sticky_actions']:
#         return Action.Sprint
#     # We always control left team (observations and actions
#     # are mirrored appropriately by the environment).
#     controlled_player_pos = obs['left_team'][obs['active']]
#     # Does the player we control have the ball?
#     if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:  #EXEI PAIKTIS TIN MPALA KAI EINAI STIN ARISTERI OMADA
#         # Shot if we are 'close' to the goal (based on 'x' coordinate).
#         if controlled_player_pos[0] > 0.5:
#             return Action.Shot
#         # Run towards the goal otherwise.
#         return Action.Right
#     else:
#         # Run towards the ball.
#         if obs['ball'][0] > controlled_player_pos[0] + 0.05:
#             return Action.Right
#         if obs['ball'][0] < controlled_player_pos[0] - 0.05:
#             return Action.Left
#         if obs['ball'][1] > controlled_player_pos[1] + 0.05:
#             return Action.Bottom
#         if obs['ball'][1] < controlled_player_pos[1] - 0.05:
#             return Action.Top
#         # Try to take over the ball if close to the ball.
#         return Action.Slide

In [ ]:
# # Set up the Environment.
# from kaggle_environments import make
# env = make("football", configuration={"save_video": True, "scenario_name": "11_vs_11_kaggle", "running_in_notebook": True})
# output = env.run(["/kaggle/working/submission.py", "do_nothing"])[-1]
# print('Left player: reward = %s, status = %s, info = %s' % (output[0]['reward'], output[0]['status'], output[0]['info']))
# print('Right player: reward = %s, status = %s, info = %s' % (output[1]['reward'], output[1]['status'], output[1]['info']))
# env.render(mode="human", width=800, height=600)